In [7]:
include("../LiPoSID.jl")

using LinearAlgebra
function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

function LME_operator_symbolic(H, J)
    
    Id = I(2)   
    U = -im*(Id ⊗ H - transpose(H) ⊗ Id)
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 
    
    Lᴸᴹᴱ = U + D
    
    return Lᴸᴹᴱ
end

function LME_operator_from_DMD(A)

    M = [ 0  0   1  1
          1  im  0  0 
          1  -im  0  0
          0  0  -1  1 ]/2
   
    Lᴰᴹᴰ =   M * A * inv(M)
   
    return Lᴰᴹᴰ 
   
end


LME_operator_from_DMD (generic function with 1 method)

In [5]:
using SymPy
@syms s

function Transfer_Function(A, b)
    inv(I(4)*s - A)*b
end

bₓ = [1, 0, 0, 1]

4-element Vector{Int64}:
 1
 0
 0
 1

In [31]:
ω = symbols("ω", real=true)
γ = symbols("γ", real=true)

Hᴸᴹᴱ = [ ω        0
         0        0   ]


Jᴸᴹᴱ = [ 0     γ
         0     0. + 0im  ]

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)

Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)

4-element Vector{Sym{PyCall.PyObject}}:
 (1.0*s + 0.5*γ^2)/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
             1.0*ω/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
                                1.0*γ^2/(1.0*s^2 + 1.0*s*γ^2)
                                                          1/s

In [33]:
data_dir = "../DATA/"
γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

8-element Vector{String}:
 "0.0795"
 "0.2513"
 "0.7947"
 "2.5133"
 "7.9477"
 "25.133"
 "79.477"
 "251.33"

In [34]:
γᵢ = γ_list[1]
γᶠ = parse(Float64, γᵢ)

ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

Lᴰᴹᴰ = LME_operator_from_DMD(Aᴰᴹᴰ⁻ˢᴮ)

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

@assert abs(LiPoSID.frobenius_norm2(inv(M) * Lᴰᴹᴰ * M - Aᴰᴹᴰ⁻ˢᴮ)) < 1e-9


Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bₓ)

Gᴰᴹᴰ = Transfer_Function(Aᴰᴹᴰ⁻ˢᴮ, bₓ)

ΔG = together.(Gᴸᴹᴱ - Gᴰᴹᴰ)



4-element Vector{Sym{PyCall.PyObject}}:
 ((1.0*s + 0.5*γ^2)*(0.0537351296273617*s^11 - 0.537431989493472*s^10 + 2.49216215181927*s^9 - 7.06810246375873*s^8 + 13.6161501187972*s^7 - 18.7019932699156*s^6 + 18.6858479380257*s^5 - 13.5809059221676*s^4 + 7.03761848028421*s^3 - 2.47711317682608*s^2 + 0.533257512491634*s - 0.0532245088838165) - (1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)*(8.40339421429779e-9*s^9 - 5.22597907985769e-8*s^8 + 1.36153243753955e-7*s^7 - 1.76564903096445e-7*s^6 + 7.64305263764925e-8*s^5 + 1.04634520295214e-7*s^4 - 1.95329288960175e-7*s^3 + 1.44143086172718e-7*s^2 - 5.42239231345288e-8*s + 8.6131354845674e-9) - (1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)*(0.0537351296273617*s^10 - 0.490341361615125*s^9 + 2.04999919273456*s^8 - 5.16884432333996*s^7 + 8.70122396289053*s^6 - 10.2157670513666*s^5 + 8.4698497450774*s^4 - 4.89604715832827*s^3 + 1.88831597905184*s^2 - 0.43876030391626*s + 0.0466361891845671))/((1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)*(0.0537351296273617*s^

In [37]:
ΔG[3]

    ⎛     2⎞ ⎛                       7                        6               
- s⋅⎝s + γ ⎠⋅⎝- 8.78955490306472e-7⋅s  + 6.13270807510381e-6⋅s  - 1.8618820584
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

         5                        4                        3                  
0851e-5⋅s  + 3.19913696278969e-5⋅s  - 3.36699875284782e-5⋅s  + 2.1742993771043
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

      2                                            ⎞     ⎛     2⎞ ⎛           
5e-5⋅s  - 7.99091990309037e-6⋅s + 1.291612031916e-6⎠ - s⋅⎝s + γ ⎠⋅⎝0.000177280
──────────────────────────────────────────────────

In [40]:
poly = expand.(numerator.(ΔG))[1]

                      11  2                       11                       10 
- 0.0268675648136809⋅s  ⋅γ  - 0.0470906362817416⋅s   - 0.0134337824068404⋅s  ⋅

 4                      10  2                       10  2                     
γ  + 0.221625358464995⋅s  ⋅γ  - 0.0537351296273617⋅s  ⋅ω  + 0.442163011344493⋅

 10                      9  4                     9  2                      9 
s   + 0.122585338302933⋅s ⋅γ  - 0.80391806456514⋅s ⋅γ  + 0.490341353211731⋅s ⋅

 2                     9                      8  4                     8  2   
ω  - 1.89925827657201⋅s  - 0.512499785118693⋅s ⋅γ  + 1.63479295530735⋅s ⋅γ  - 

                  8  2                     8                     7  4         
2.04999914047477⋅s ⋅ω  + 4.91492633247159⋅s  + 1.29221104679668⋅s ⋅γ  - 1.8931

            7  2                     7  2                    7                
4872692702⋅s ⋅γ  + 5.16884418718672⋅s ⋅ω  - 8.4862262949795⋅s  - 2.17530594658

     6  4                      6  2           

In [42]:
degree(poly, s)

11

In [49]:
poly.coeff(s)

                   4                      2                      2            
0.109690089535046⋅γ  + 0.219992558448114⋅γ  + 0.438760358140183⋅ω  - 0.0532245

         
088838165

In [50]:
poly.coeff(s^2)

                    4                      2                     2            
- 0.47207903079873⋅γ  - 0.799796230272855⋅γ  - 1.88831612319492⋅ω  + 0.4866213

        
14693931

In [ ]:
xpanded_poly.args[2]

In [ ]:
f.args[1]

In [ ]:
@syms x
fraction = x / 2
numerator_fraction = numerator(fraction)

In [ ]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

max_negs = []
sum_negs = []
num_negs = []

for γᵢ in γ_list 

    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

end